In [ ]:
class PositionCrawlerHelper(object):

    def __init__(self, step, node, G):
        self._graph = G
        self._node = node
        self._step = step
        
        self._direct_succers = []
        self._subdirect_succers = []

    def successors(self):
        return (self.only_direct_successors(), self.only_subdirect_successors())
    
    def only_direct_successors(self):
        self.categorize_successors()
        return self._direct_succers
    
    def only_subdirect_successors(self):
        self.categorize_successors()
        return self._subdirect_succers
    
    def categorize_successors(self):
        succers = self._graph[self._node['label']]
        
        if (len(succers) > 0) and (not self._direct_succers):
            for node in succers:
                wg = self._graph.nodes[node].get('weight')

                diff = wg - self._step
                if diff is 1:
                    self._direct_succers.append(node)
                else:
                    self._subdirect_succers.append(node)
                    
        
    
class PositionCrawler(object):
    def __init__(self, node, grid, G, index, Helper=PositionCrawlerHelper):
        self._grid = grid
        self._node = node
        self._step = node.get('weight')
        self._index = index
        self._graph = G
        
        self._gpos = GPos()
        self._options = {
           'max_inter': 30
        }
        
        self._helper = Helper(self._step, node, G)
     
    def get_gpos(self):
        return self._gpos
    
    def defaults(self, args=None):
        self._gpos.set_position((self._default_x(), self._default_y()))
        
    def _default_y(self):
        return self._max_empty_y(self._step)
        
    def _max_empty_y(self, step):
        if step not in self._grid:
            return 0
        
        return max(self._grid[step], key=int) + 1

    def has_pos(self, x, y):
        return (x in self._grid) and (y in self._grid[x])
      
    def _default_x(self):
        return self._step
    
class PositionCrawlerPattern(PositionCrawler):
        
    def soft_balance(self):
        start_y = self._default_y()

        nstep = self._step + 1
        nposy = self._max_empty_y(nstep)
        
        if start_y > nposy:
            diff = start_y - nposy
            for i in range(diff):
                posy = nposy + i
                self._gpos.append_dummy((nstep, posy))
    
    def child_balance(self, ):
        succers, subsuccers = self._helper.successors()
        succers_size = len(succers)
        
        succers_size += 1 if len(subsuccers) >= 1 else 0 # if chess hourse is actived, put one more line

        if succers_size >= 2:
            self.balance_nodes(succers_size-1)
            self.grow_node(succers_size)
     
    def chess_hourse(self):
        succers = self._helper.only_subdirect_successors()
        
        if len(succers) >= 1:
            posy = self._max_empty_y(self._step)
            
            for item in succers:
                subnode = self._graph.nodes[item]
                
                w = subnode.get('weight')
                switch_y = self.chess_hourse_eligible_y(subnode, w)
                diff = ((w+switch_y) - self._step)
                self.chess_hourse_dummie(diff, posy)
                

    def chess_hourse_dummie(self, diff, posy):
        for rg in range(diff):
            nstep = self._step + rg + 1

            if not self.has_pos(nstep, posy):
                self._gpos.append_dummy((nstep, posy))
    
    def chess_hourse_find_y(self, y1, y2, step, node):
        found = False
        
        for it in range(y2-1, y1-1, -1):
            if (step in self._grid) and (self._grid[step].get(it, '-') is not '-'):
                if self._grid[step][it] is node:
                    found = True
                break
        return found
    
    def chess_hourse_recursive_y(self, step):
        found = step
        while True:
            check = self._max_empty_y(step)
            if check <=1:
                break
            step += 1
        return step
        
    def chess_hourse_eligible_y(self, subnode, w):
        label = subnode.get('label')
        diff = 0
        
        if label in self._index:
            mypos = self._index[label]
            eligible_y = self._max_empty_y(w)
            
            last_node_in_line = self.chess_hourse_find_y(mypos[1], eligible_y, w, label)
            if not last_node_in_line:
                nstep = self.chess_hourse_recursive_y(w+1)
                self._gpos.append_switch(label, (nstep-1, mypos[1]))
                diff = nstep - w
            
        return diff 
        
    def grow_node(self, total):
        start_y = self._default_y()
        
        for ps in range(total-1):
            nps = (self._step, start_y + (ps+1))
            self._gpos.append_dummy(nps)
            self._gpos.inc_size()
            
    def balance_nodes(self, qtd):
    
        for nl in range(self._step):
            last = self._max_empty_y(nl)

            for np in range(qtd):
                posy = last + np
                self._gpos.append_dummy((nl, posy))
                
    def set_position(self):
        start_y = self._default_y()
        self._gpos.set_position((self._default_x(), start_y))
            
            
            
class PositionCrawlerIterator(object):
    
    def __init__(self, Pattern):
        self._pattern = Pattern
        
    def map(self):
        return ['chess_hourse','soft_balance', 'child_balance', 'set_position']
        
    def find_rule(self):
        for check in self.map():
            getattr(self._pattern, check)()
    
    
class PositionFinder(object):
    def __init__(self, G, gridhist=GraphHistogram):
        
        self._grid = {}
        self._index = {}
        self._graph = G
        
        GridHistogram = gridhist(G)
        self._nmax = GridHistogram.max_value()
        self._hist = GridHistogram.get_counter()

    def set_grid(self, grid):
        self._grid = grid
        
    def set_index(self, index):
        self._index = index
        
    def give_pos(self, node, CPattern=PositionCrawlerPattern, CIterator=PositionCrawlerIterator):        
        Patterns = CPattern(node, self._grid, self._graph, self._index)
        CIterator(Patterns).find_rule()
        
        return Patterns.get_gpos()

    
    
print("-------------------------")
Orchestration = GridOrchestrator(G)
Orchestration.create(entries)

Layout = DrawLayout(Orchestration.get_grid())
Layout.draw_nodes()
Layout.draw_connections(G.edges())
Layout.save()

print(Orchestration.get_grid().get_grid())
print(Orchestration.get_grid().index)
   
display(SVG(filename='test.svg'))

In [ ]:
class GPos(object):
    
    def __init__(self):
        self._dummies = []
        self._positions = []
        self._switch = []
        self._size = 1
        
    def append_dummy(self, dummy):
        self._dummies.append(dummy)
        
    def append_swap(self, swap):
        self._swaps.append(swap)
    
    def set_position(self, pos):
        self._positions = pos
    
    def append_switch(self, ifrom, ito):
        self._switch.append((ifrom, ito))
        
    def set_size(self, size):
        self._size = size
        
    def inc_size(self, inc=1):
        self._size += inc
    
    def get_size(self):
        return self.get_positions()[3]
    
    def get_x(self):
        return self.get_positions()[0]
    
    def get_y(self):
        return self.get_positions()[1]
    
    def get_positions(self):
        return self._get('positions')
    
    def get_dummies(self):
        return self._get('dummies')
    
    def get_swap(self):
        return self._get('swap')
    
    def _get(self, item):
        prop = '_'+item
        data = getattr(self, prop)
        return data
    
class GridMap(object):
    def __init__(self, finder):
        self.clean()
        
        self.finder = finder
        self.finder.set_grid(self.grid)
        self.finder.set_index(self.index)
    
    def add_pos_grid(self, node):
        gpos = self.finder.give_pos(node)
        label = node.get('label')
        
        if isinstance(gpos, GPos):
            if gpos._dummies:
                self.create_dummies(gpos._dummies, label)
                
            if gpos._positions:
                self.create_positions(gpos._positions, label, gpos._size)
                
            if gpos._switch:
                self.make_switchs(gpos._switch)
            
        return gpos
    
    def create_dummies(self, pos, label):
        if isinstance(pos, (tuple)):
            self.add_dummy(pos)

        if isinstance(pos, (list)):
            for p in pos:
                self.add_dummy(p)
                
    def create_positions(self, pos, label, size=1):
        self.index[label] = (*pos, size)
        return self._add_grid(*pos, label)
    
    def make_switchs(self, switchs):
        if isinstance(switchs, (list)):
            for swt in switchs:
                print(swt)
                self.make_switch(*swt)
    
    def make_switch(self, switch, npos):
        pos = self.index[switch]
        print("=== ", switch)
        print(npos)
        del self.grid[pos[0]][pos[1]]
        del self.index[switch]
        
        self.create_positions(npos, switch)
        
    def add_dummy(self, pos):  
        return self._add_grid(*pos, '-')
    
    def _add_grid(self, x, y, item):
        if x not in self.grid:
            self.grid[x] = {}
            
        self.grid[x][y] = item
        return (x, y)

    def get_pos(self, item):
        if item in self.index:
            return self.index[item]
    
    def get_grid(self):
        return self.grid  
    
    def clean(self):
        self.grid = {}
        self.index = {}